# Practice of CIFAR10

In [2]:
import tensorflow as tf

print(tf.__version__)

2.2.0-dev20200315


In [ ]:
# Implementation practice of AlexNet in Sequential mode
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers, Sequential
from tensorflow.keras.utils import plot_model
import os


(x,y), (x_test, y_test) = datasets.cifar10.load_data()
print(x.shape, y.shape, x_test.shape, y_test.shape)

# remove the last division of y and y_test
y = tf.squeeze(y, axis=-1)
y_test = tf.squeeze(y_test, axis=-1)
print(x.shape, y.shape, x_test.shape, y_test.shape)

def preprocess(x,y):
    x = 2*tf.cast(x,dtype=tf.float32)/255.-1
    y = tf.cast(y,dtype=tf.int32)
    return x,y
    
# need to construct Training object with the imported tensors
train_db = tf.data.Dataset.from_tensor_slices((x,y))
# shuffle and preporcess and setup batch
train_db = train_db.shuffle(1000).map(preprocess).batch(128)

test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.map(preprocess).batch(128)

alexnet_layers = [
    # Layer 1
    layers.Conv2D(filters = 96, kernel_size = [11,11], strides = 4, padding = [[0,0],[2,2],[2,2],[0,0]], activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[3,3], strides=2, padding = 'valid'),
    # Layer 2
    layers.Conv2D(filters = 256, kernel_size = [5,5], strides = 1, padding = [[0,0],[2,2],[2,2],[0,0]], activation=tf.nn.relu),
    layers.MaxPool2D(pool_size = [3,3], strides = 2, padding = 'valid'),
    # Layer 3
    layers.Conv2D(filters = 384, kernel_size = [3,3], strides = 1, padding = [[0,0],[1,1],[1,1],[0,0]], activation=tf.nn.relu),
    # Layer 4
    layers.Conv2D(filters = 384, kernel_size = [3,3], strides = 1, padding = [[0,0],[1,1],[1,1],[0,0]], activation=tf.nn.relu),
    # Layer 5
    layers.Conv2D(filters = 256, kernel_size = [3,3], strides = 1, padding = [[0,0],[1,1],[1,1],[0,0]], activation=tf.nn.relu),
    layers.MaxPool2D(pool_size = [3,3], strides=1, padding = 'same'),
    
    # Before going to FC layers, a batch normal layer is needed.
    layers.BatchNormalization(),
    layers.Flatten(),
    
    # FC layers
    # Layer 6
    layers.Dense(4096, activation = tf.nn.relu),
    layers.Dropout(rate = 0.2),
    # Layer 7
    layers.Dense(4096, activation = tf.nn.relu),
    layers.Dropout(rate = 0.2),
    # Layer 8
    layers.Dense(10, activation=None)
]


def main():
    
    alexnet = Sequential(alexnet_layers)
    alexnet.build(input_shape = [None, 32, 32, 3])
    alexnet.summary()
    plot_model(alexnet, 'alex.png')
    #vgg_layers = []
    
    optimizer = optimizers.Adam(lr = 1e-4)
    
    for epoch in range(100):
        for step, (x, y) in enumerate(train_db):
            with tf.GradientTape() as tape:
                out = alexnet(x)
                y_onehot = tf.one_hot(y, depth = 10)

                loss = tf.losses.categorical_crossentropy(y_onehot, out, from_logits = True)
                loss = tf.reduce_mean(loss)

            grads = tape.gradient(loss, alexnet.trainable_variables)
            optimizer.apply_gradients(zip(grads, alexnet.trainable_variables))

            if step % 100 == 0:
                print(epoch, step, 'loss:', float(loss))

        total_num = 0
        total_correct = 0
        for x,y in test_db:
            out = alexnet(x)
            prob = tf.nn.softmax(out, axis = 1)
            pred = tf.argmax(prob, axis = 1)
            pred = tf.cast(pred, dtype = tf.int32)

            correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
            correct = tf.reduce_sum(correct)

            total_num += x.shape[0]
            total_correct += int(correct)

        acc = total_correct / total_num
        print(epoch, 'acc:', acc)
      
    print("Iteration finished!!")
 
if __name__ == '__main__':
    main()

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            multiple                  34944     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_6 (Conv2D)            multiple                  614656    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_7 (Conv2D)            multiple                  885120    
_________________________________________________________________
conv2d_8 (Conv2D)            multiple                  1327488   
____

In [4]:
import nvidia_smi

nvidia_smi.nvmlInit()

handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
# card id 0 hardcoded here, there is also a call to get all available card ids, so we could iterate

info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)

print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

nvidia_smi.nvmlShutdown()

Total memory: 11553341440
Free memory: 11046354944
Used memory: 506986496
